## CNV Profiling

## scRNA-Seq Melanoma dataset

In [ ]:
library(Seurat)


In [ ]:
setwd("/QRISdata/Q4386/QIMR_workshop_CNV/Mel_3samples")

In [ ]:
Mel_scRNA<-readRDS("Mel_3samples_75pcs.rds")
Mel_scRNA

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
Idents(Mel_scRNA)<-Mel_scRNA@meta.data$cell_type
DimPlot(Mel_scRNA)

## CopyKAT

In [ ]:
## copyKAT installation
#install.packages("devtools")
#library(devtools)
#install_github("navinlabcode/copykat")
library(copykat)

## Run CopyKAT

In [ ]:
exp.rawdata<-read.csv("MPSs_count_mat.txt",sep="\t")
rownames(exp.rawdata)<-exp.rawdata$GENE
exp.rawdata<-exp.rawdata[,-1]

In [ ]:
exp.rawdata[1:5,1:5]

In [ ]:
copykat.test <- copykat(rawmat=exp.rawdata, id.type="S", ngene.chr=5, win.size=25, KS.cut=0.1, sam.name="test", distance="euclidean", norm.cell.names="",output.seg="FLASE", plot.genes="TRUE", genome="hg20",n.cores=1) #hg20 built-in copkat is the hg38 coords


### Read CopyKAT predictions

In [ ]:
p<-read.csv("test_copykat_prediction.txt", sep="\t",header=TRUE)
head(p)

In [ ]:
colnames(Mel_scRNA)<-gsub("-1",".1",colnames(Mel_scRNA))
seurat_colnames<-colnames(Mel_scRNA)
rownames(p)<-p$cell.names	
p<-p[seurat_colnames,]
# Assuming infercnv_sub_groupings is your DataFrame and seurat_colnames contains the column names from your Seurat object

# Find the order of the BC values in the seurat_colnames
bc_order <- match(p$cell.names, seurat_colnames)

# Reorder the columns of infercnv_sub_groupings based on the order
p_reordered <- p[order(bc_order), ]

# View the reordered DataFrame
dim(p_reordered)
head(p_reordered)

## Identify Malignant Tumor cells

In [ ]:
Mel_scRNA@meta.data$copykat_pred<-p$copykat.pred
#Mel_scRNA@meta.data$copykat_aneuploid <- ifelse(Mel_scRNA@meta.data$copykat_pred == "aneuploid", "Aneuploid", "")
aneuploid_cells <- rownames(Mel_scRNA@meta.data)[Mel_scRNA@meta.data$copykat_pred == "aneuploid"]

DimPlot(Mel_scRNA, group.by = "copykat_pred")
DimPlot(Mel_scRNA, group.by = "copykat_pred", cells.highlight = aneuploid_cells, cols.highlight = "red")

## Identify the sample with malignant tumor

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)
DimPlot(Mel_scRNA, group.by = "copykat_pred", split.by = "orig.ident")
DimPlot(Mel_scRNA, group.by = "copykat_pred", cells.highlight = aneuploid_cells, cols.highlight = "red", split.by = "orig.ident")


## InferCNV

In [ ]:
library(infercnv)

In [ ]:
options("Mel_scRNA.assay.version" = "v3")
infercnv_meta <- data.frame(V1 = Mel_scRNA@meta.data$cell_type)
rownames(infercnv_meta)<-rownames(Mel_scRNA@meta.data)
head(infercnv_meta)


In [ ]:
unique(Mel_scRNA@meta.data$Level1_res1)

### Run InferCNV

In [ ]:
infercnv_obj = CreateInfercnvObject(
  raw_counts_matrix=GetAssayData(Mel_scRNA, slot="counts"),
  annotations_file=infercnv_meta,
  delim="\t",
  gene_order_file="/scratch/project/stseq/Prakrithi/skin_atlas/infercnv/hg38_gencode_v27.txt",
  ref_group_names=NULL) # set list of confident normal cells matching ur anno file c("KC","Immune","Fibroblast","Endothelial cell"))

In [ ]:
out_dir = tempfile()
infercnv_obj_default = infercnv::run(
    infercnv_obj,
    cutoff=1, # cutoff=1 works well for Smart-seq2, and cutoff=0.1 works well for 10x Genomics
    out_dir="infercnv/all/",
    cluster_by_groups=TRUE, 
    plot_steps=FALSE,
    denoise=TRUE,
    HMM=FALSE,
    no_prelim_plot=TRUE,
    png_res=60
)


In [ ]:
infercnv_sub_groupings<-read.csv("infercnv/all/infercnv_subclusters.observation_groupings.txt", sep=" ",header=TRUE)
tail(infercnv_sub_groupings)

In [ ]:
unique(infercnv_sub_groupings$Annotation.Group)
dim(infercnv_sub_groupings)

In [ ]:
# Assuming infercnv_sub_groupings is your DataFrame and seurat_colnames contains the column names from your Seurat object
rownames(infercnv_sub_groupings)<-infercnv_sub_groupings$BC

# Find the order of the BC values in the seurat_colnames
bc_order <- match(rownames(infercnv_sub_groupings), colnames(Mel_scRNA))

# Reorder the columns of infercnv_sub_groupings based on the order
infercnv_sub_groupings_reordered <- infercnv_sub_groupings[order(bc_order), ]

# View the reordered DataFrame
head(infercnv_sub_groupings_reordered)

In [ ]:
# Assuming Mel_scRNA is your Seurat object and infercnv_sub_groupings is your data frame

# Create a new column in Mel_scRNA@meta.data with empty values
Mel_scRNA@meta.data$infercnv_pred <- ""

# Get the row names of both data frames
mel_row_names <- rownames(Mel_scRNA@meta.data)
infercnv_row_names <- rownames(infercnv_sub_groupings)

# Find the matching row names
matching_row_names <- intersect(mel_row_names, infercnv_row_names)

# Update the new column with "Aneuploid" for matching rows
Mel_scRNA@meta.data[matching_row_names, "infercnv_pred"] <- "Aneuploid"

# Check the updated meta.data
head(Mel_scRNA@meta.data)


In [ ]:
infercnv_aneuploid_cells <- rownames(Mel_scRNA@meta.data)[Mel_scRNA@meta.data$infercnv_pred == "Aneuploid"]
DimPlot(Mel_scRNA, group.by = "infercnv_pred", cells.highlight = infercnv_aneuploid_cells, cols.highlight = "red")


In [ ]:
# Create a new column in Mel_scRNA@meta.data with empty values
Mel_scRNA@meta.data$copykat_infercnv_consistent <- ""

# Update the new column with "Aneuploid" where both conditions are met
Mel_scRNA@meta.data$copykat_infercnv_consistent[Mel_scRNA@meta.data$infercnv_pred == "Aneuploid" & Mel_scRNA@meta.data$copykat_pred == "aneuploid"] <- "Aneuploid"

# Highlight the cells that are consistent between infercnv and copykat predictions
consistent_cells <- rownames(Mel_scRNA@meta.data)[Mel_scRNA@meta.data$copykat_infercnv_consistent == "Aneuploid"]
DimPlot(Mel_scRNA, group.by = "copykat_infercnv_consistent", cells.highlight = consistent_cells, cols.highlight = "red")
